In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df_credits =pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df_movies  =pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

actor_list = ["Sean Connery", "John Travolta", "VinceVaughn", "Terry Crews",
              "John Malkovich", "Billy Zane", "Ben Kingsley", "Ving Rhames",
              "Patrick Stewart", "Jason Statham", "Tyrese Gibson", "Bruce Willis",
              "Stanley Tucci", "Vin Diesel", "Dwayne Johnson", "Samuel L. Jackson"]




In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
df_movies['id'] = df_movies['id'].apply(convert_int)
df_movies[df_movies['id'].isnull()]
df_credits['id'] = df_credits['movie_id'].apply(convert_int)

df = df_movies
df = df.merge(df_credits, on='id')
df['cast'] = df['cast'].apply(ast.literal_eval)
df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [ ]:
df.head()

In [ ]:
s = df.apply(lambda x: pd.Series(x['cast']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'actor'
cast_df = df.drop('cast', axis=1).join(s)
cast_revenue = cast_df.groupby('actor')['revenue'].sum()
cast_revenue.head()

In [ ]:
cast_revenue.loc(actor_list)

In [ ]:
actor_list = ["Sean Connery", "John Travolta", "Vince Vaughn", "Terry Crews",
              "John Malkovich", "Billy Zane", "Ben Kingsley", "Ving Rhames",
              "Patrick Stewart", "Jason Statham", "Tyrese Gibson", "Bruce Willis",
              "Stanley Tucci", "Vin Diesel", "Samuel L. Jackson", "Yul Brynner",
              "Michael Jordan", "Howie Mandel", "Damon Wayans", "Phil McGraw",
              "Jason Alexander", "Peter Boyle", "J.K. Simmons", "Dwayne Johnson"]
sorted_actors = []
for actor in actor_list:
    sorted_actors.append({'actor':actor, 'gross':cast_revenue[actor]})
for i, row in enumerate(sorted(sorted_actors, key=lambda item: -item['gross'])):
    print('{:2d} {:18,.2f} - {}'.format(i+1, row['gross'], row['actor']))